<a href="https://colab.research.google.com/github/SrijaniChakrabarti/Prediction-System-/blob/master/networkanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy
import pandas as pd


In [1]:
# Link travel time = free flow time * ( 1 + B * (flow/capacity)^Power ).
# Link generalized cost = Link travel time + toll_factor * toll + distance_factor * distance
# The network files also contain a "speed" value for each link. In some cases the "speed" values are consistent with the free flow times, in other cases they represent posted speed limits, and in some cases there is no clear knowledge about their meaning. All of the results reported below are based only on free flow travel times as described by the functions above, and do not use the speed values.

In [4]:
netfile = '/content/sample_data/Chicagosketchnet.tntp'
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)


In [5]:
net

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,547,49500,0.86267,0.00,0.15,4,0,0,3
1,2,548,49500,0.86267,0.00,0.15,4,0,0,3
2,3,549,49500,0.86267,0.00,0.15,4,0,0,3
3,4,550,49500,0.86267,0.00,0.15,4,0,0,3
4,5,551,49500,0.86267,0.00,0.15,4,0,0,3
...,...,...,...,...,...,...,...,...,...,...
2945,931,906,500,22.65430,14.78,0.15,4,0,0,1
2946,932,386,49500,0.86267,0.00,0.15,4,0,0,3
2947,932,515,5000,10.44260,5.93,0.15,4,0,0,2
2948,933,387,49500,0.86267,0.00,0.15,4,0,0,3


In [6]:
net.head()

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,547,49500,0.86267,0.0,0.15,4,0,0,3
1,2,548,49500,0.86267,0.0,0.15,4,0,0,3
2,3,549,49500,0.86267,0.0,0.15,4,0,0,3
3,4,550,49500,0.86267,0.0,0.15,4,0,0,3
4,5,551,49500,0.86267,0.0,0.15,4,0,0,3


In [8]:
net.columns

Index(['init_node', 'term_node', 'capacity', 'length', 'free_flow_time', 'b',
       'power', 'speed', 'toll', 'link_type'],
      dtype='object')

In [9]:
len(net)


2950

In [10]:
net.columns[0]

'init_node'

In [11]:
# taking all edges of graph in list
init_node_list = list(net.init_node)
term_node_list = list(net.term_node)
t = len(net)
edge_list = []
i = 0
while i < t:
  term_temp = term_node_list[i]
  init_temp = init_node_list[i]
  edge_list.append((init_temp,term_temp))
  i += 1


In [19]:
# Functions for constructing adjacency list
adj_list = {}
mylist = []
def add_node(node):
  if node not in mylist:
    mylist.append(node)
  else:
    print("Node ",node," already exists!")
 
def add_edge(node1, node2):
  temp = []
  if node1 in mylist and node2 in mylist:
    if node1 not in adj_list:
      temp.append(node2)
      adj_list[node1] = temp
   
    elif node1 in adj_list:
      temp.extend(adj_list[node1])
      temp.append(node2)
      adj_list[node1] = temp
       
  else:
    print("Nodes don't exist!")
  



      

In [20]:
# giving all the inputs for constructing adjacency list
for i in init_node_list:
  add_node(i)
for l in term_node_list:
  add_node(l)
for (ini,term) in edge_list:
  add_edge(ini,term)


Node  388  already exists!
Node  388  already exists!
Node  388  already exists!
Node  389  already exists!
Node  389  already exists!
Node  390  already exists!
Node  390  already exists!
Node  391  already exists!
Node  391  already exists!
Node  391  already exists!
Node  392  already exists!
Node  392  already exists!
Node  392  already exists!
Node  393  already exists!
Node  393  already exists!
Node  393  already exists!
Node  394  already exists!
Node  394  already exists!
Node  394  already exists!
Node  395  already exists!
Node  395  already exists!
Node  395  already exists!
Node  396  already exists!
Node  397  already exists!
Node  397  already exists!
Node  397  already exists!
Node  398  already exists!
Node  398  already exists!
Node  398  already exists!
Node  399  already exists!
Node  399  already exists!
Node  399  already exists!
Node  400  already exists!
Node  400  already exists!
Node  400  already exists!
Node  401  already exists!
Node  401  already exists!
N

In [18]:
len(mylist) # mylist is list of all nodes without duplicates

933

In [17]:
adj_list # dictionary for tracking adjacent nodes

{1: [547],
 2: [548],
 3: [549],
 4: [550],
 5: [551],
 6: [552],
 7: [553],
 8: [554],
 9: [555],
 10: [556],
 11: [557],
 12: [558],
 13: [559],
 14: [560],
 15: [561],
 16: [562],
 17: [563],
 18: [564],
 19: [565],
 20: [566],
 21: [567],
 22: [568],
 23: [569],
 24: [570],
 25: [571],
 26: [572],
 27: [573],
 28: [574],
 29: [575],
 30: [576],
 31: [577],
 32: [578],
 33: [579],
 34: [580],
 35: [581],
 36: [582],
 37: [583],
 38: [584],
 39: [585],
 40: [586],
 41: [587],
 42: [588],
 43: [589],
 44: [590],
 45: [591],
 46: [592],
 47: [593],
 48: [594],
 49: [595],
 50: [596],
 51: [597],
 52: [598],
 53: [599],
 54: [600],
 55: [601],
 56: [602],
 57: [603],
 58: [604],
 59: [605],
 60: [606],
 61: [607],
 62: [608],
 63: [609],
 64: [610],
 65: [611],
 66: [612],
 67: [613],
 68: [614],
 69: [615],
 70: [616],
 71: [617],
 72: [618],
 73: [619],
 74: [620],
 75: [621],
 76: [622],
 77: [623],
 78: [624],
 79: [625],
 80: [626],
 81: [627],
 82: [628],
 83: [629],
 84: [630],
 

In [22]:
# Priority Queue implementation in Python

class PriorityQueue:
  # Function to heapify the tree
  def heapify(arr, n, i):
    # Find the largest among root, left child and right child
    largest = i
    l = 2 * i + 1
    r = 2 * i + 2

    if l < n and arr[i] < arr[l]:
        largest = l

    if r < n and arr[largest] < arr[r]:
        largest = r

    # Swap and continue heapifying if root is not largest
    if largest != i:
        arr[i], arr[largest] = arr[largest], arr[i]
        heapify(arr, n, largest)
   
   # Function to insert an element into the tree
   
  def insert(array, newNum):
    size = len(array)
    if size == 0:
        array.append(newNum)
    else:
        array.append(newNum)
        for i in range((size // 2) - 1, -1, -1):
            heapify(array, size, i)
  
  # Function to delete an element from the tree
  def deleteNode(array, num):
    size = len(array)
    i = 0
    for i in range(0, size):
        if num == array[i]:
            break

    array[i], array[size - 1] = array[size - 1], array[i]

    array.remove(size - 1)

    for i in range((len(array) // 2) - 1, -1, -1):
        heapify(array, len(array), i)
